In [ ]:
# inspired from
# https://www.kaggle.com/sinamhd9/keras-models-tensorflow-data-dataset-tpu-part1

In [ ]:
!pip install tensorflow_io

In [ ]:
%reset -f
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# ML tools 
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow import keras
from kaggle_datasets import KaggleDatasets
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Activation, Conv2D, MaxPooling2D, Dropout, Conv2D,MaxPooling2D,GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from tensorflow.keras.applications import Xception
import tensorflow_io as tfio
import os
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras import models, layers
from tensorflow.keras.initializers import RandomUniform, lecun_normal, GlorotUniform
import gc
import cv2

from tensorflow.keras.optimizers import Adam
from tensorflow import data, io, image, reshape, float32
from tensorflow.random import set_seed
from numpy.random import seed
import random
img_size = 512
TPU_FLAG = 0

# change
seeding = 1
# mse/Kl_divergence

In [ ]:
train = pd.read_csv('../input/vinbigdata-chest-xray-abnormalities-detection/train.csv')
train

In [ ]:

df_target = pd.DataFrame()
df_target['image_id'] = train['image_id'].unique()+ '.dicom'
df_target['label'] = 1
df_target['height'] = -1
df_target['width'] = -1
df_target.index = df_target['image_id']
df_target

In [ ]:
# Convert DICOM to PNG via openCV
%time
inputdir = '../input/vinbigdata-chest-xray-abnormalities-detection/train/'
outdir = 'train/'
os.mkdir('train')
# test_list = [os.path.basename(x) for x in glob.glob(inputdir + './*.dcm')]

def read_dicom(path):
    image_bytes = tf.io.read_file(path)
    image = tfio.image.decode_dicom_image(
        image_bytes, 
        dtype = tf.uint16
    )
    image = tf.squeeze(image, axis = 0)
    im_shape = image.shape
    image = tf.image.resize(image, (img_size, img_size), )
    
    image = image - tf.reduce_min(image)
    image = image / tf.reduce_max(image)
    image = tf.cast(image * 255, tf.uint8)
    return image, im_shape

count = 0
for f in df_target.index:   
    img, im_shape = read_dicom(inputdir + f) # read dicom image
    df_target['height'].loc[f] = im_shape[0]
    df_target['width'].loc[f] = im_shape[1]
    df_target['image_id'].loc[f]  = f.replace('.dicom','.png')
    cv2.imwrite(outdir + f.replace('.dicom','.png'),img.numpy()) # write png image
    count = count+1
    if count%100 == 0:
        print(count)
    
im_shape

In [ ]:
df_target.to_csv('train_df_height_width', index=False)